In [47]:
import cv2
import numpy as np
from itertools import count
from skimage.measure import ransac
from skimage.transform import FundamentalMatrixTransform, EssentialMatrixTransform

vid_path = '5.hevc'

def generate_frames(vid_path):
    video = cv2.VideoCapture(vid_path, cv2.CAP_FFMPEG)
    _, prev_frame = video.read()
    for t in count():
      ret, curr_frame = video.read()
      if not ret:
        break
      yield prev_frame, curr_frame
      prev_frame = curr_frame
    video.release()
    cv2.destroyAllWindows()

def extractFeatures(frame):
  orb = cv2.ORB_create()
  # only works on b/w images
  pts = cv2.goodFeaturesToTrack(np.mean(frame,axis=2).astype(np.uint8), 3000, 0.01, minDistance=7)
  # we need kp class to feed it into ORB.compute to get descriptors
  kps = [cv2.KeyPoint(x=f[0][0], y=f[0][1], size=10) for f in pts]
  kps, des = orb.compute(frame, kps)
  return np.array([(kp.pt[0], kp.pt[1]) for kp in kps]), des

def bfmatcher(kps, dess):
  bf = cv2.BFMatcher(cv2.NORM_HAMMING)
  # find closest descriptors between frames, hamming norm for ORB (BRIEF, BRISK...)
  # they are binary string types
  # L1, L2 for SIFT/SURF
  matches = bf.knnMatch(dess[0], dess[1], k=2)
  res = []
  # DMatch obj: distance (the lower the better)
  # trainIdx: index of the descriptor in train desc
  # queryIdx: index of the descriptor in query desc
  # imgIdx: index of the train image
  for m,n in matches:
    # Lowe's ratio test
    # https://stackoverflow.com/questions/51197091/how-does-the-lowes-ratio-test-work
    if m.distance < 0.75 * n.distance:
        kp1 = kps[0][m.queryIdx]
        kp2 = kps[1][m.trainIdx]
        res.append((kp1, kp2))
  res = np.array(res)
  # prune the outliers by fitting F or E (random sampling of the data)
  assert len(res)>=8, 'not enough points'
  model, inliers = ransac((res[:,0],
                          res[:,1]),
                          FundamentalMatrixTransform, 
                          #EssentialMatrixTransform,
                          min_samples=8,
                          residual_threshold=0.5, 
                          max_trials=1000)
  #print (f'good kps: {len(res[inliers])/len(res)*100:.2f}% out of {len(res)}')
  return res[inliers, 0], res[inliers, 1]

w = 1164
h = 874
F = 910

K = [[F, 0, w//2],
     [0, F, h//2],
     [0, 0, 1]]

def normalize(x):
  x[:,0] -= w//2
  x[:,1] -= h//2  
def denormalize(x):
  x[:,0] += w//2
  x[:,1] += h//2

In [46]:
for p, c in generate_frames(vid_path):

  kps_1, des_1 = extractFeatures(p)
  kps_2, des_2 = extractFeatures(c)

  p1, p2 = bfmatcher([kps_1, kps_2], [des_1, des_2])
  
  for p in p2:
    cv2.circle(c, (int(p[0]), int(p[1])), 1, (255,255,0)) 
  
  for k1, k2 in zip(p1, p2):
    cv2.line(c, tuple(k1.astype(int)), tuple(k2.astype(int)), (0,200,200), 1)
  
  cv2.imshow('v', c)
  key = cv2.waitKey(1)
  if key == ord('q'):
    break
  
cv2.destroyAllWindows()

good kps: 43.91% out of 271
good kps: 65.24% out of 187
good kps: 66.16% out of 198
good kps: 63.79% out of 174
good kps: 69.32% out of 176
good kps: 68.00% out of 200
good kps: 68.34% out of 199
good kps: 62.84% out of 183
good kps: 71.49% out of 228
good kps: 77.35% out of 181
good kps: 64.19% out of 148
good kps: 70.00% out of 150
good kps: 63.64% out of 176
good kps: 71.69% out of 166
good kps: 60.30% out of 199
good kps: 67.65% out of 204
good kps: 68.51% out of 181
good kps: 61.62% out of 198
good kps: 63.11% out of 206
